In [1]:
import gwaslab as gl

In [ ]:
mysumstats = gl.Sumstats(
    "/mnt/hdd_1/ofgeha/galaxy-gwas-tools/Data/21001_T2T_liftover_rsid.tsv.gz",
    snpid="SNPID",
    chrom="CHR",
    pos="POS",
    ea="EA",            # In your data, it's now EA (was ALT)
    nea="NEA",          # In your data, it's now NEA (was REF)
    eaf="EAF",          # In your data, it's now EAF (was minor_AF/Frq)
    beta="BETA",
    se="SE",
    p="P",
    # Note: 'Dir' and 'N' are not in your column list, 
    # so we should omit them or point to the correct ones.
    # n="n_complete_samples", # Optional: if you want to use sample size
    verbose=True,
    readargs={'sep': '\t'}
)

2026/01/10 12:33:12 GWASLab v4.0.4 https://cloufield.github.io/gwaslab/
2026/01/10 12:33:12 (C) 2022-2026, Yunye He, Kamatani Lab, GPL-3.0 license, gwaslab@gmail.com
2026/01/10 12:33:12 Python version: 3.12.3 (main, Nov  6 2025, 13:44:16) [GCC 13.3.0]
2026/01/10 12:33:13 Start to initialize gl.Sumstats from file :/mnt/hdd_1/ofgeha/galaxy-gwas-tools/Data/21001_T2T_liftover_rsid.tsv.gz
